### **INTRODUCTION TO MOBILE ROBOTICS PROJECT**
##### MICRO-452

#### **Authors** : 
##### Laure Coquoz
##### Hendrik Hilsberg
##### Nathan Kammoun
##### Adrien O'Hana

# Table of contents 

* [0. Introduction](#Intro)
* [I. Computer Vision](#c1)
* [II. Global Navigation](#c2)
* [III. Local Navigation](#c3)
* [IV. Kalman Filter](#c4)

# 0. Introduction <a class="anchor" id="Intro"></a>

We started our project by thinking about the design of our map. We decided to go with a white map, whose corners are represented by apriltags. We didn't define the edges of the map as a limit for the Thymio, and assumed that we would place the obstacles in a way that the Thymio always has enough space to pass between an obstacle and the edge of the map. Concerning the obstacles and the goal, we chose red cardboard pieces of various shapes for the former and a green dot for the latter. Finally, to localize the robot, we used a different apriltag than the one from the edges and placed it on top of the robot.

Regarding the code, the main function is separated in two blocks: one for vision, and the other one for filtering and motion combined. We used a counter to define the ratio of executions of one block with regard to other. The vision block will be executed once, followed by ten executions of the block containing filtering and motion. These values were not chosen randomly : we timed each block to get an idea of the execution times. While the vision lasts approximately 150ms in the worst cases, the filtering and motion part combined last up to around 10ms. Using the above mentioned ratio means that there is a new vision step approximately four times every second. 

Here is a pseudo-code for a better comprehension of the main function:

# I. Vision <a class="anchor" id="c1"></a>

### Field Detection 

We wanted to keep the setup as easy to work with as possible. We knew we would have to re-install the whole setup a lot so we opted for the most flexible option in terms of camera positionning. Placing the camera on top of the field seems like the most straight-forward option but it also brings a lots of constraints. On top of that, we had to make sure that the webcam gives a consistent output each time we reinstalled the setup. That is why we opted for a convenient and flexible option in terms of webcam positionning : field detection. 

The field refers to the space where the thymio can go. We delimited it with four april tags at each corners. April tag is a visual fiducial system used in robotics, the AprilTag detection software computes the precise 3D position and orientation of the tags.

A black color filtering is done on the image before corner detection in order to make AprilTag detection more efficient. Once the april tags are detected, the four corners positions are registered in order to later crop and transform each of the camera outputs. 

<p float="left">
  <img src="images/CameraFrame.jpeg" width="500" />
</p>

The detection of the corners is done in the **detect_corner_tags()** function in our code, using the **Detector** object of the **apriltag** librairy, the tags we used for the corners were the 36h11.

The position of the four corners are passed to the **getPerspectiveTransform()** and **warpPerspective()** functions from **OpenCV** which results in a perspective transformed subset of the camera frame. Next, the result is resized to fit the real dimensions of the field. Our field size is **1050mmx1188mm**, we reisze the image to be of size **1050x1188** in order to get a 1 pixel/mm ratio. 

<p float="left">
  <img src="images/Field.jpeg" width="400" />
</p>

*Bellow you can see the code of the main function used for field detection. The code access the **crop** attribute which contains the positions of each corners of the field* 

In [7]:
def get_field(self, image) : 

    """
    Goal:  Crop the camera frame to fit the thymio field 
    Input:  image = camera frame 
    Output: field = cropped and transformed camera frame to fit the thymio field 
    """

    # Field Size 
    width  = self.field_size['width']
    height = self.field_size['height']

    # Image 
    rows,cols,ch = image.shape
    pts1 = np.float32([self.crop['top_left'], self.crop['top_right'], self.crop['bottom_left'], self.crop['bottom_right']])
    pts2 = np.float32([[0,0],[width, 0],[0, height],[width,height]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    field = cv2.warpPerspective(image, M, (width,height))

    return field 

### Thymio Detection 

We used another AprilTag to detect the thymio. The good thing being that april tag detection gives us the position of each one of the detected april tags corners. We compute the orientation of the tag by considering the angle between its center and the middle point between its top left and top right corners.

*Note : The output of the thymio detection is the position of the thymio tag. In the Robot class the actual position of the thymio is computed by translating the thymio tag center position along its orientation.*

*Bellow you can see the code of the main function used in thymio detection* 

In [5]:
def thymio_detection(self, field) : 
    """
    Goal:  Detect the thymio tag  
    Input: field = cropped and transformed camera frame to fit the thymio field 
    Output: thymio tag = dictionnary with positions of thymio tag center and 'nose' 
            -- The thymio tag 'nose' is the middle position between its top right and top left corners --
    """
    
    # Thymio Tag Position 
    thymio_tag = {'pos' : None, 'nose': None}

    # Tag Detection 
    options  = apriltag.DetectorOptions(families="tag25h9")
    detector = apriltag.Detector(options)
    results  = detector.detect(cv2.cvtColor(field, cv2.COLOR_BGR2GRAY)) 

    # Results 
    if len(results)==0 : 
        print('Thymio Not Detected')
        return None
    else :   
        r = results[0]
        (ptA, ptB, ptC, ptD) = r.corners
        ptB = (int(ptB[0]), int(ptB[1]))
        ptC = (int(ptC[0]), int(ptC[1]))
        ptD = (int(ptD[0]), int(ptD[1]))
        ptA = (int(ptA[0]), int(ptA[1]))

        thymio_tag['pos']  = (int(r.center[0]), int(r.center[1]))
        thymio_tag['nose'] = (int((ptA[0]+ptB[0])/2), int((ptA[1]+ptB[1])/2))
        return thymio_tag

### Obstacle Detection

In order to plan a path for the robot we need to compute the position of each one of the obstacle within the field, to be precise we need the positions of each corners of each obstacles.  

The obstacles were chosen to be red pieces of paper on the field. We first perform a filtering on red values which filters the obstacles on the thymio field image. The *red_filtering* function filters all values within a given RGB range (the threshold are set manually given obstacles color).

<p float="left">
  <img src="images/RedFiltering.jpeg" width="300" />
</p>

One thing we had to take into account is that we only detect the thymio center, which does not reflect the size of the entire robot. The stategy we used was to increase the obstacle size to account for the actual size of the robot. This is done through a sliding kernel on the filtered red values, it results in larger obstacles. This is done thanks to the **dilate()** function from OpenCV. 

A combination of two functions is used to compute the corners of each augmented obstacles. The **findContours()** function computes the contours of obstacles given the dilated output of the red filtering pass. Then the **approxPolyDP()** function fits a polygon from the contours and outputs the corners of these polygons. 

*Note : The detection of rounded shapes is not a problem as they are just interprated as polygons with a high number of corners*

*Bellow you can see the code of the main function used for obstacles detection* 

In [4]:
def obstacles_detection(self, field, thymio_clearance=120) :     
        """
        Goal:  Detected obstacles 
        Input: field = cropped and transformed camera frame to fit the thymio field 
               thymio_clearance = how much the obstacles should be increased considering the thymio size (mm) 
        Output: obstacles_dict = dictionnary of obstacles corners positions 
                dilated_obstacles_dict = dictionnary of dilated obstacles corners positions 
        """
        # Store Obstacle Shapes
        obstacles_dict = {}
        dilated_obstacles_dict = {}
        
        # Get obstacles 
        red_filter = self.red_filtering(field)
        
        # Bitwise Inverse
        obstacles = cv2.bitwise_not(red_filter[:, :, 0])
        
        # Expand Obstacles
        kernel = np.ones((thymio_clearance, thymio_clearance), np.uint8)
        dilated_obstacles = cv2.dilate(obstacles, kernel, iterations=1)
        
        # Find contours of filtered shapes
        contours, _  = cv2.findContours(obstacles, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        dilated_contours, _ = cv2.findContours(dilated_obstacles, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Compute corners from contours 
        for i, contour in enumerate(contours):
            # Approximate Shape 
            approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
            obstacles_dict[i] = [(corners[0][0], corners[0][1]) for corners in approx]
         
        # Compute corners from contours
        for j, contour in enumerate(dilated_contours):
            # Approximate Shape 
            approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
            dilated_obstacles_dict[j] = [(corners[0][0], corners[0][1]) for corners in approx]
            
        return obstacles_dict, dilated_obstacles_dict

### Goal Detection 

The goal was chosen to be a green piece of paper of any shape. First, a filtering on green values was performed to filter the goal on the thymio field, the *green_filtering* function filters all values within a given RGB range (the threshold are set manually given goal color).

The **findContours()** function computes the contours of the goal, then the **moments()** function enables to find the center of the goal contour and make it the detected goal point.  

*Bellow you can see the code used for goal detection* 

In [2]:
def goal_detection(self, field) : 
    """
    Goal:  Detected obstacles 
    Input: field = cropped and transformed camera frame to fit the thymio field 
    Output: goal = dictionnary containing position of the goal, position is set to origin in case of detection failure
    """
    # Goal Position 
    goal = {}

    # Filtering of green values
    green_filter = self.green_filtering(field)
    contours, _  = cv2.findContours(green_filter, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Test detection 
    if len(contours) == 1  : 
        print('Goal Not Detected')
        goal['pos'] = (0, 0)
        return goal
    else :          
        # Get center of green shape
        M = cv2.moments(contours[1])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"]) 
        goal['pos'] = (cX, cY)

        return goal

# II. Global Navigation <a class="anchor" id="c2"></a>

### Global Path Planning

The global path is computed once as soon as we detect the first thymio position. The goal is detected and obstacle detection is performed : we have a goal point, a list of corners for each obstacles as well as the robot position. The global path planning approach we chosed was to compute the shortest path thanks to a visibility graph.

The visibily graph was easily computed thanks to the **PyVisGraph** librairy.

*Below the code used to compute the global path*

In [ ]:
def path_planning(self, field_map, final_goal, obstacles) : 
    """
    Goal : Calculate the shortest path between the robot and the goal
    Input : field_map 2D array with shape of the size of the map in mm, final_goal['pos'] = (x,y), obstacles dictionary 
    Output : field_map with added path
    """                                   
    # Definition 
    start = vg.Point(self.x_est, self.y_est)
    objective  = vg.Point(final_goal['pos'][0], final_goal['pos'][1])

    # Calculate shortest path 
    polys = [[vg.Point(corners[0], corners[1]) for corners in obstacle] for (i, obstacle) in obstacles.items()]
    g = vg.VisGraph()
    g.build(polys)
    shortest = g.shortest_path(start, objective)

    # Define Path and Next Goal in Graph  
    self.path = shortest
    self.next_goal = 1

    # Update Field Map 
    field_map = self.draw_path(field_map)

    return field_map

### Following the way 

The path planning gives the thymio a road map it has to follow, a series of goal it has to reach and between which it ideally has to navigate in straight lines. The thymio updates the position of its next goal when reaching one until it reaches the final destination. As it is not likely that the robot position coincides exactly with the goal position, the updates happens once the thymio is sufficiently close from its goal. 

*Below the code used to handle navigation*

In [ ]:
def update_next_goal(self, tolerance=30, verbose=True) : 
    """
    Goal : Check distance with current goal and updates it if it comes sufficiently close
    Output : Return True if Goal Updated and False if not 
    """         
    # Get next waypoint and calculate its distance to the robot
    goal_pos = self.path[self.next_goal]
    distance2goal = math.sqrt((self.x_est-goal_pos.x)**2+(self.y_est-goal_pos.y)**2)

    # Update the next waypoint if the robot is close enough to the next waypoint
    if distance2goal < tolerance : 
        if verbose : print('Reached Goal {0}'.format(self.next_goal))
        if self.next_goal == len(self.path)-1 : 
            self.finished=True
            if verbose : print('Your journey is over little thymio')
        else :
            self.next_goal =  self.next_goal+1 
            if verbose : print('Now directing towards {0}'.format(self.next_goal))
        return True 
    else :
        return False 

We thought about stopping when reaching each intermediate goal and turning towards the next one before moving but we opted for a more dynamic behavior. Our strategy allows the thymio to deviate from the planned path while constantly redirecting itself towards its next goal. 

- Green Lines: Planned path 
- Yellow Dots : Intermediate goals 
- Orange Line : Thymio to goal direction

<img src="images/GlobalNav.png" width="600" height="400">

The deviation is computed by computing the difference between the thymio orientation and the direction towards its next goal. Then the deviation angle is multiplied by a deviation gain that is used to update the motors input.

$ L_{speed_{goal}} = Gain_{deviation} * deviation\_angle$  
$ R_{speed_{goal}} = Gain_{deviation} * deviation\_angle$

*Below the code used to compute the deviation angle*

In [ ]:
def compute_deviation(self) : 
    '''
    Goal : Compute angle difference between thymio and goal to use it in naviguation
    Output : Deviation angle (rad)
    '''
    #Calculate angle between robot and next waypoint
    goal_pos = self.path[self.next_goal]
    omega = math.atan2(goal_pos.y-self.y_est, goal_pos.x-self.x_est)       

    #Calculate difference with robot orientation and adjust angle to be in the interval [-pi,pi]
    if omega-self.theta_est<-np.pi :
        dev=-(omega-self.theta_est+np.pi)
    elif omega-self.theta_est>np.pi : 
        dev=-(omega-self.theta_est-np.pi)
    else :
        dev=omega-self.theta_est
        
    return dev

### Real Time Vizualisation

As a way to check that everything is working correctly we implemented a real time vision of the field. 
This additional feature aims at providing a better insight on what is going on during the robot's journey. First, field detection is performed, then obstacles and goal detection follow. As soon as the thymio is detected, global path planning is performed and the shortest path is added to the map. 

Once this initial mapping is done, we update the map at a given frequency to update the robot's position along the way. 

- The green dot shows the goal detected position 
- The orange dots and lines represents the augmented obstacles
- The **orange** dots represent each one of the point that the robots has to reach along the way. 
- The **yellow** lines represent the straight lines the robot has to follow to reach the next goal-point. 
- The blue dot represents the position od the thymio tag if it is detected 
- The pink line shows its orientation 
- The pink dot shows the position of the robot

<p float="left">
  <img src="images/Visualisation.jpeg" width="400" />
</p>

# III. Local Navigation <a class="anchor" id="c3"></a>

There is a simple way to add local navigation to the existing structure of the project without having to implement a state machine : **Potential Field Navigation**. 
The idea is to compute speed components for the left and right wheels depending on the front sensor values. Those speed components are then added to the ones calucated to reach the next waypoint. Therefore, if there is a new obstacle in the way the robot will simply adjust its trajectory to avoid the obstacle while still going towards the next waypoint. 

Additionaly we've implemented a form of memory so that the robot remembers that it is avoiding a local obstacle even when the sensors have just stopped seeing it. In order to do this the previous left and right wheel speed components for local avoidance are added to the new ones with a decay, so eventually these components reach null values shortly after there is no more obstacle in sight.  

Here are the formulas implemented the code below :

$ L_{speed} = L_{speed_{goal}} + L_{speed_{obstacle}} $  
$ R_{speed} = R_{speed_{goal}} + R_{speed_{obstacle}} $

$ L_{speed_{obstacle}} = prox\_values_{front} \cdot weight\_matrix_{left} $  
$ R_{speed_{obstacle}} = prox\_values_{front} \cdot weight\_matrix_{right} $

As seen in the practicals, we extend the array containing the proximity sensor values (**x**) so it can also record the previous values for left and right speeds due to the obstacle, and that is the memory. These values stored in **x[5]** and **x[6]** are divided by a number greater than 1 we call **mem_decay** so the memory component diminishes with time.

In [ ]:
deviation = robot.compute_deviation()
prox_values = get_prox_values()

#speed to reach goal
L_speed_goal = speed + devGain * deviation
R_speed_goal = speed - devGain * deviation

#store front sensor values
for i in range(5):
    # Get and scale inputs
    x[i] = prox_values[i]//sensor_scale

#Memory
x[5] = L_speed_obstacle//mem_decay
x[6] = R_speed_obstacle//mem_decay

#Reset obstacle speeds after storing
L_speed_obstacle = 0
R_speed_obstacle = 0

#compute outputs of neurons
for i in range(len(x)):    
    L_speed_obstacle += x[i] * w_l[i]
    R_speed_obstacle += x[i] * w_r[i]

#sum speeds to reach goal and to avoid obstacle
L_speed = L_speed_goal + L_speed_obstacle
R_speed = R_speed_goal + R_speed_obstacle

#apply new speed to motors
set_motors(int(L_speed), int(R_speed))
robot.L_speed = L_speed
robot.R_speed = R_speed

An example of local obstacle avoidance can be seen in the following video : https://drive.google.com/drive/folders/1v1ewrE3aYPwxLCQqBAzRLtnIOmm1f-SC

# IV. Kalman filter <a class="anchor" id="c4"></a>

When the robot is moving on the map, it uses the speed measurements of its motors and the position given by the webcam to localize itself. Unfortunately this data is subject to noise which can induce error. Indeed, the speeds that we get from the wheel motors are not exactly the ones that we set due to perturbations on the measurements and due to the nature of the physical world. Additionally the state that we capture with the webcam can not be considered as ground truth and has an uncertainty due to discretization. 

In order to better estimate the position and orientation of the robot, sensor Fusion can be done. This can be achieved with the use of a Kalman Filter which will furthermore allow the robot to localize itself even if one of the data sources is momentarily missing.

In this project, we choose to use the Extended Kalman Filter that is a Kalman Filter applied to a non linear system linearized at different discrete time. It uses measurements observed over time, statistical noise and other inaccuracies of the system to produce an estimation of the state. With these estimations, it is possible to be more accurate than by trusting only on the measurements. 

The Kalman Filter uses two different phases: the prediction phase and the measurement update phase. While we have no new measurement given by the camera we use the prediction step. If we have just received a new measurement from the camera, we use the update step. These steps give an estimation of the state of the robot that we have chosen to define as $(x, y, \theta)$.

The following snippet shows the conditions on which one step or the other is done, which depends on the boolean **boolKF** set to **True** every time a new state is retrieved from the camera.

In [ ]:
#------------------------ FILTERING ---------------------------

# When the vision block is not "ON"
else :
    # Get the value of the motors of the Thymio
    [L_speed,R_speed] = get_motors()      

    if boolKF is True:
        # Last block was the vision 
        if thymio_tag is None:
            # The Thymio tag is not detected so we predict the Thymio's state from the odometry
            kalman.kalman_prediction(robot,L_speed,R_speed)
            # Indicate that last block was not vision (Kalman_prediction)
            boolKF = False
        else:
            # The Thymio tag is detected so we can update the Thymio's state with the webcam
            # Indicate that last block was not vision (Kalman_update)
            boolKF = kalman.kalman_update(robot)

    elif boolKF is False:
        # Last block was the vision so we predict the Thymio's state with the webcam
        kalman.kalman_prediction(robot,L_speed,R_speed)
        # Indicate that last block was not vision (Kalman_prediction)
        boolKF = False


    #Update Goal 
    robot.update_next_goal(tolerance=10)   

### Prediction

The prediction phase is applied when the system does not have the value of the measurement of webcam. We use the webcam as the ground truth and when we do not have access to it, the Kalman Filter produces an estimation of the current state variables and its uncertainties. The state is predicted from the odometry formulas (in matrix **B**). We use the data from motion sensors to estimate the change in position over time. The formula used is:
-  $E = AE+BU \cdot deltaT$

Where E is the state vector containing $(x, y, \theta)$, A is the transition matrix of the state (Identity here), B is the transition matrix of the input based on the odometry formulas, U the motor speeds of the left and the right wheels and deltaT is the time that passed between the last Kalman prediction.

Once the prediction of the state is done, this phase update also the uncertainties of the system. The formula used is the following:
- $Q = BU_{var}B^T$
- $P = APA^T + Q$

Where Q contains the uncertainties due to the speed of the motors, $U_{var}$ is the variation of the speed of the motors and P contains the uncertainties of the system. The uncertainty of the system P is used in the second phase, the measurement update. 

The variances of the left and right speeds was calculated with our own robot giving it a fixed speed of 50 (our operating speed) and measuring the resultings left and right speeds many times over a long period. The result of this experiment gave us a rounded value of 19 which is then converted to _mm_ using **B**.

Note : For better tracking in the direction perpendicular to the wheels, we decided to add additional uncertainty as a practical fix. This explains the addition of an identity matrix in the formula computing Q, which actually adds uncertainty in the three state variables in case we want to pick up the robot and displace it. In the following videos, we show how the robot is tracked with and without this additional uncertainty.  (https://drive.google.com/drive/folders/1iKniy9w9gv823rdwRVa8SnVvJ6L8U6Rl)

Below, you can see the code used to perform the prediction function.

In [ ]:
def kalman_prediction(self,thymio,L_speed,R_speed):

    """
    Goal:     Since at this point we do not have the measurement done by the webcam, it
              predicts the state of the robot based on the odometry and it updates the 
              variances of the system.
    Input:    thymio = thymio class with the data of our robot
              L_speed = speed of the left wheel applied until now
              R_speed = speed of the right wheel applied until now
    Output:   -
    """

    # Compute the time between the last update/prediction and this time
    deltaT = time.time_ns()/10e8 - self.lastKalman
    # Compute the transition matrix of the input corresponding to the odometry formula
    self.B = np.matrix([[0.5*math.cos(self.E[2])*self.c,0.5*math.cos(self.E[2])*self.c],
                        [0.5*math.sin(self.E[2])*self.c,0.5*math.sin(self.E[2])*self.c],
                        [1/self.b*self.c, -1/self.b*self.c]],dtype= 'float')

    # Compute the predicted state of the robot (AE+BU)
    self.E = np.dot(self.A, self.E) + np.dot(self.B, self.U)*deltaT

    # Compute the additionnal uncertainty due to the motors
    Q = np.dot(self.B, np.dot(self.U_var_m,self.B.T))+np.eye(3)
    # Update the variance of the system
    self.P = np.dot(np.dot(self.A,self.P),np.transpose(self.A))+Q

    # Update the state with the predicted x, y and theta
    thymio.x_est = self.E[0].item()
    thymio.y_est = self.E[1].item()
    thymio.theta_est = self.E[2].item()

    # Keep the speeds of the robot in U to compute the next prediction
    self.U = np.matrix([[L_speed],[R_speed]],dtype= 'float')

    # Update the time of the last kalman done to find deltaT
    self.lastKalman = time.time_ns()/10e8

### Measurement update

When the measurement of the webcam is done, the Kalman filter directly applies its measurement update function. Using the new state value given by the camera and the uncertainty matrix P computed in the previous function, it corrects the state of the robot. The formulas used in this phase are:
- $K= (P*H^T) / (H*P*H^T +R)$
- $E = E+(K*(Z-H*E))$
- $P = (I-K*H)*P$

Where H is the measurement matrix, here the identity because the webcam data has already been converted to state variables. K is the Kalman gain that specifies the degree to which the measurement is incorporated into the new estimation of the state. R is the matrix with the uncertainties of the measurements. 

In this project, the uncertainty of the x and y values is the size of one pixel (1mm) and 1 degree for $\theta$. P is then updated here since we have to add the uncertainties of the measurement. 

Below, you can see the code used to perform the measurement update function.

In [ ]:
def kalman_update(self,thymio):
    """
    Goal:   Since we have the measurement of the camera, it can update 
            the state of the robot and update the variances of the system.
    Input:  thymio = thymio class with the data of our robot

    Output: returns false to set the boolean that says that the last block 
            was not the vision block.
    """

    # Put the measurement of the webcam in the measured state matrix
    Z = np.matrix([[thymio.x_cam],[thymio.y_cam],[thymio.theta_cam]],dtype= 'float')

    # Update the state of the kalman with the state of the robot
    self.E[0] = thymio.x_est 
    self.E[1] = thymio.y_est 
    self.E[2] = thymio.theta_est 

    # Computation of the Kalman gain 
    K_den = np.linalg.inv(np.dot(self.H,np.dot(self.P,np.transpose(self.H))) + self.R)
    K_num = np.dot(self.P,np.transpose(self.H))
    K = np.dot(K_num,K_den)        

    #Correction of the state with the Kalman gain and the measurements
    self.E = self.E + np.dot(K,(Z-np.dot(self.H,self.E)))

    #Update of the variance of the system
    I = np.eye(3)
    self.P = np.dot((I-np.dot(K,self.H)),self.P)

    # Update the state with the corrected x, y and theta
    thymio.x_est = self.E[0].item()
    thymio.y_est = self.E[1].item()
    thymio.theta_est = self.E[2].item()

    # Update the time of the last kalman done to find deltaT
    self.lastKalman = time.time_ns()/10e8

    return False

## Conclusion & Demonstration

Coordinating and joining the differents parts of this project has been a real challenge and we are proud to have successful results that we have demonstrated and recorded in the link below. 

In these videos we can see the robot and the map through a phone camera as well as the extracted information from the vision which has been screen recorded on the computer. We demonstrate global navigation through multiple obstacles and local navigation with an added obstacle. We also demonstrate that the estimation of the Kalman filter is working even when the camera is hidden for a short time.

https://drive.google.com/drive/folders/1_lI-Vve-SCC8VqV3Ai6MmwPxx2l8Qoub?usp=sharing

## Sources : 

#### Apriltag : 
https://april.eecs.umich.edu/software/apriltag#:~:text=AprilTag%20is%20a%20visual%20fiducial,tags%20relative%20to%20the%20camera. 

#### OpenCV :
https://opencv.org/ 

#### Pyvisgraph : 
https://github.com/TaipanRex/pyvisgraph 


